## Repository configuration

In [ ]:
# Allows getting information about GPU
!nvidia-smi

In [ ]:
# Clone project from your GitHub repository (replace <TOKEN>, <REPOSITORY_URL>, <REPOSITORY_NAME> with your own values)

# To get token, please follow the instructions from:
# https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens#creating-a-fine-grained-personal-access-token
!git clone -b solution https://<TOKEN>@github.com/<REPOSITORY_URL>
%cd "<REPOSITORY_NAME>"

In [ ]:
# Install requirements
!pip install -r requirements.txt
!pip3 install torch torchvision

## Session configurations

In [ ]:
import os

from easydict import EasyDict
from torchvision import transforms

from utils.enums import WeightsInitType, LayerType

In [ ]:
# neptune.ai logger environment variables (replace <TOKEN>, <EXPERIMENTS_DIR> with your own values)
os.environ['NEPTUNE_API_TOKEN'] = '<TOKEN>'
os.environ['NEPTUNE_DATA_DIRECTORY'] = '<EXPERIMENTS_DIR>'

### Dataset configurations

In [ ]:
ROOT_DIR = '/kaggle/input'

data_cfg = EasyDict()

# Path to the directory with dataset files
data_cfg.path_to_data = os.path.join(ROOT_DIR, 'emotion_detection')
data_cfg.annot_filename = 'data_info.csv'

# Label mapping
data_cfg.label_mapping = {
    'angry': 0,
    'disgusted': 1,
    'fearful': 2,
    'happy': 3,
    'neutral': 4,
    'sad': 5,
    'surprised': 6
}
data_cfg.classes_num = 7

# Training configuration
data_cfg.train_transforms = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,), std=(0.5,))
])

data_cfg.eval_transforms = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,), std=(0.5,))
])


### Model configurations

In [ ]:
model_cfg = EasyDict()

# Layers configuration
model_cfg.layers = [
    {'type': LayerType.Linear, 'params': {'in_features': 32 * 32, 'out_features': 128, 'bias': True}},
    {'type': LayerType.ReLU, 'params': {}},
    {'type': LayerType.Dropout, 'params': {'p': 0.2}},
    {'type': LayerType.Linear, 'params': {'in_features': 128, 'out_features': 64, 'bias': True}},
    {'type': LayerType.ReLU, 'params': {}},
    {'type': LayerType.Dropout, 'params': {'p': 0.2}},
    {'type': LayerType.Linear, 'params': {'in_features': 64, 'out_features': 7, 'bias': True}},
]

# Weights and bias initialization
model_cfg.params = EasyDict()
model_cfg.params.init_type = WeightsInitType.xavier_uniform
# More details about initialization methods parameters can be found here: https://pytorch.org/docs/stable/nn.init.html
model_cfg.params.init_kwargs = {}
model_cfg.params.zero_bias = True

### Experiment configurations

In [ ]:
# Replace <REPOSITORY_NAME>, <PROJECT_NAME>, <EXPERIMENT_NAME> with your own values
OUTPUT_DIR = '/kaggle/working/<REPOSITORY_NAME>'

experiment_cfg = EasyDict()
experiment_cfg.seed = 0
experiment_cfg.num_epochs = 20

# Train parameters
experiment_cfg.train = EasyDict()
experiment_cfg.train.batch_size = 64
experiment_cfg.train.learning_rate = 1e-3
experiment_cfg.train.continue_train = False
experiment_cfg.train.checkpoint_from_epoch = None

# Overfit parameters
experiment_cfg.overfit = EasyDict()
experiment_cfg.overfit.num_iterations = 500

# Neptune parameters
experiment_cfg.neptune = EasyDict()
experiment_cfg.neptune.env_path = os.path.join(ROOT_DIR, '.env')
experiment_cfg.neptune.project = '<PROJECT_NAME>'
experiment_cfg.neptune.experiment_name = '<EXPERIMENT_NAME>'
experiment_cfg.neptune.run_id = None
experiment_cfg.neptune.dependencies_path = os.path.join(ROOT_DIR, 'requirements.txt')

# MLflow parameters
experiment_cfg.mlflow = EasyDict()
experiment_cfg.mlflow.tracking_uri = 'http://127.0.0.1:5000'
experiment_cfg.mlflow.experiment_name = ''
experiment_cfg.mlflow.run_id = None
experiment_cfg.mlflow.dependencies_path = os.path.join(ROOT_DIR, 'requirements.txt')

# Checkpoints parameters
experiment_cfg.checkpoints_dir = os.path.join(OUTPUT_DIR, 'experiments', experiment_cfg.neptune.experiment_name, 'checkpoints')
experiment_cfg.checkpoint_save_frequency = 1
experiment_cfg.checkpoint_name = 'checkpoint_%s'
experiment_cfg.best_checkpoint_name = 'best_checkpoint'

# Data parameters
experiment_cfg.data = data_cfg

# Model parameters
experiment_cfg.model = model_cfg

## Training and evaluation

In [ ]:
import pandas as pd

from dataset.emotions_dataset import EmotionsDataset
from executors.trainer import Trainer
from torch.utils.data import DataLoader
from utils.enums import SetType

In [ ]:
# Trainer initialization
trainer = Trainer(experiment_cfg)

In [ ]:
# Training
trainer.fit()

In [ ]:
# Make predictions

# Initialiaze trainer instance if new session
# trainer = Trainer(experiment_cfg, init_logger=False)

# Get data to make predictions on
test_dataset = EmotionsDataset(data_cfg, SetType.test, transforms=data_cfg.eval_transforms)
test_dataloader = DataLoader(test_dataset, experiment_cfg.train.batch_size, shuffle=False)

# Get predictions
model_path = experiment_cfg.best_checkpoint_name
predictions, image_paths = trainer.predict(model_path, test_dataloader)

# Save results to submission file
test_results_df = pd.DataFrame({'ID': image_paths, 'prediction': predictions})
test_results_df.to_csv('test_predictions.csv', index=False)

## Some notes

In [ ]:
# For long-time running experiments use 'Save & Run All' mode. From documentation:
# Save & Run All creates a new session with a completely clean state and runs your notebook from top to bottom.
# This is perfect for major milestones or when you want to share your work, as it gives you (and anyone else who reads your notebook) the confidence that your notebook can be run reproducibly.
# In order to save successfully, the entire Notebook must execute within 12 hours (9 hours for TPU notebooks).
# Save & Run All is identical to the “Commit” behavior you may have used previously on Kaggle.

# GPU is turned off by default. To use GPU accelerator at the backend:
# When run the notebook as usually: Session options-> accelerator -> GPU P100
# When run in "Save & Run All" mode: Save version -> version type (Save & Run All (Commit) -> advanced settings -> "Run with GPU for this session"